# Tutorial: Machine Learning in scikit-learn
*From the [Sebastian Raschka](https://github.com/rasbt/pattern_classification) the GitHub repository [pattern_classification](https://github.com/rasbt/pattern_classification)*

![Machine learning](images/01_robot.png)

# Part1. Encoding Categorical Features in Classification Tasks

Features can come in various different flavors. Typically we distinguish between

- continuous and
- categorical (discrete)
features.


And the categorical features can be categorized further into:

- ordinal and
- nominal (= no order implied) features.



Now, most implementations of machine learning algorithms require numerical data as input, and we have to prepare our data accordingly. This notebook contains some useful tips for how to encode categorical features using Python pandas and scikit-learn.

## Example Data

First, let us create a simple example dataset with 3 different kinds of features:
- pclass: ordinal
- sex, embarked, survived: nominal
- age: continuous

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/titanic_curso.csv', sep=',')
df.head()

## Nominal Features. 

**Feature Survived. Label**

"Typical" machine learning algorithms handle class labels with "no order implied" - unless we use a ranking classifier (e.g., SVM-rank). Thus, it is save to use a simple set-item-enumeration to convert the class labels from a string representation into integers.

In [ ]:
df['survived'].unique()

In [ ]:
label_mapping = {label:idx for idx,label in enumerate(set(df['survived']))}

df['survived'] = df['survived'].map(label_mapping)
df.head()

In [ ]:
df['survived'].unique()

**Feature Sex**

However, we can use another simply trick and convert the "sex" and "embarked" into binary features: Each possible value becomes a feature column itself (with values 1 or 0).

In [ ]:
sex_mapping = {
           'male': (0,1),
           'female': (1,0)}

df['sex'] = df['sex'].map(sex_mapping)
df.head()

**Feature Embarked**

In [ ]:
df['embarked'].unique()

In [ ]:
embarked_mapping = {
           'Southampton': (0,0,1),
           'Cherbourg': (0,1,0),
           'Queenstown': (1,0,0)}

df['embarked'] = df['embarked'].map(embarked_mapping)
df.head()

This also can be done with a sklearn preprocessing method call *OneHotEncoder*

## Ordinal Features

Ordinal features need special attention: We have to make sure that the correct values are associated with the corresponding strings. Thus, we need to set-up an explicit mapping dictionary:

In [ ]:
df['pclass'].unique()

In [ ]:
pclass_mapping = {
           '1st': 2,
           '2nd': 1,
           '3rd': 0}

df['pclass'] = df['pclass'].map(pclass_mapping)
print df['pclass'].unique()
df.head()

This also can be done with a sklearn preprocessing method call **LabelEncoder**

## Inverse Mapping

If we want to convert the features back into its original representation, we can simply do so my using inverted mapping dictionaries:

In [ ]:
inv_label_mapping = {v: k for k, v in label_mapping.items()}
inv_pclass_mapping = {v: k for k, v in pclass_mapping.items()}
inv_embarked_mapping = {v: k for k, v in embarked_mapping.items()}
inv_sex_mapping = {v: k for k, v in sex_mapping.items()}

df['survived'] = df['survived'].map(inv_label_mapping)
df['pclass'] = df['pclass'].map(inv_pclass_mapping)
df['embarked'] = df['embarked'].map(inv_embarked_mapping)
df['sex'] = df['sex'].map(inv_sex_mapping)
df.head()

# Part 2. Using scikit-learn and pandas features

The scikit-learn maching library comes with many useful preprocessing functions that we can use for our convenience.

### scikit LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
df['survived'] = class_le.fit_transform(df['survived'])

df.head()

The class labels can be converted back from integer to string via the `inverse_transform` method:

### scikit OneHotEncoder

Another useful tool in scikit-learn is the [`OneHotEncoder`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html). The `OneHotEncoder` takes integer columns as input. Here we are an `LabelEncoder`, we use the `LabelEncoder` first, to prepare the `embarked` column before we use the `OneHotEncoder`.

In [ ]:
embarked_le = LabelEncoder()
df['embarked'] = embarked_le.fit_transform(df['embarked'])

df.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

embarked_ohe = ohe.fit_transform(df[['embarked']].values)
embarked_ohe

### pandas GetDummies

Also, pandas comes with a convenience function to create new categories for nominal features, namely: [`get_dummies`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.reshape.get_dummies.html).
Using the get_dummies will create a new column for every unique string in a certain column:

In [ ]:
df_new = pd.get_dummies(df)
df_new.head()

Note that the `get_dummies` function leaves the numeric columns untouched, how convenient!

In [ ]:
df_new.to_csv('./data/dataset_newTitanic.csv', index=False)